# Prototyping the calculation flow to Determine Base Passing Wire Size

This notebook contains two approaches to using the functions already written into pySolarCalc to determine the minimum wire size that will provide the required ampacity. No further calcuations (voltage drop or conduit sizing) are included yet.


## Imports

In [ ]:
import graphtik
import math
import operator
import osd
import nec_tables as nec

## Functions

I threw these together as I needed them while composing the DAG. We will need to decide which ones should be kept, if the functionality should be improved, docstrings added, and tests written.

In [ ]:
def get_wire_size(current, mat, temp):
    table = nec.cable_ampacity_310_16[mat][temp]
    wire_size = osd.lookup(current, table, keys=False)
    return (wire_size, nec.cable_ampacity_310_16[mat][temp][wire_size])

In [ ]:
def check_wire_ampacity(current, size, mat, temp, derates=1.0):
    ampacity = nec.cable_ampacity_310_16[mat][temp][size]
    return (ampacity * derates) >= current

In [ ]:
def compare_wire_sizes(cond_a, cond_b):
    """
    arguments are tuples (wire_size, ampacity)
    """
    if cond_a[1] > cond_b[1]:
        return cond_a
    else:
        return cond_b

In [ ]:
def check_wire_against_ocpd(cond, ocpd):
    return cond[1] >= ocpd

## Calc flow matching the flow chart

This doesn't quite work because the ocpd check at the end requires bumping the conductor size, but the conductor size is not an input to the calculation.
Maybe, I didn't interpet the flow chart correctly?

The names used here (and in the next DAG) need refinement before adding this to pySolarCalc.

**Note: The values in the needs list are passed sequentially to the function for each operation.**

In [ ]:
graph = graphtik.compose('base_wire_size',
                         graphtik.operation(name='ccc_derate', needs=['ccc_count', 'table'], provides=['ccc_derate'])(osd.lookup),
                         graphtik.operation(name='amb_temp_derate', needs=['amb_temp', 'wire_insulation_temp'], provides=['amb_derate'])(osd.get_ambient_temp_derate),
                         graphtik.operation(name='cond_of_use_derates', needs=['ccc_derate', 'amb_derate'], provides=['total_derate'])(operator.mul),
                         graphtik.operation(name='current_per_conductor', needs=['current', 'parallel_conductors'], provides=['parallel_current'])(operator.mul),
                         graphtik.operation(name='current_terminal_check', needs=['parallel_current', 'ocpd_derate'], provides=['terminal_check_current'])(operator.truediv),
                         graphtik.operation(name='cond_use_current', needs=['parallel_current', 'total_derate'], provides=['cond_use_check_current'])(operator.truediv),
                         graphtik.operation(name='wire_size_terminal', needs=['terminal_check_current', 'wire_material', 'terminal_temp_rating'], provides=['terminal_min_wire'])(get_wire_size),
                         graphtik.operation(name='wire_size_cond_use', needs=['cond_use_check_current', 'wire_material', 'wire_insulation_temp'], provides=['cond_use_min_wire'])(get_wire_size),
                         graphtik.operation(name='compare_wire_sizes', needs=['terminal_min_wire', 'cond_use_min_wire'], provides=['min_wire_size'])(compare_wire_sizes),
                         graphtik.operation(name='get_ocpd', needs=['current', 'voltage_type', 'ocpd_derate'], provides=['ocpd'])(osd.get_ocpd),
                         graphtik.operation(name='cable_sizing_ocpd', needs=['ocpd'], provides=['cable_sizing_ocpd'])(osd.get_cable_sizing_ocpd),
                         graphtik.operation(name='ocpd_check', needs=['min_wire_size', 'cable_sizing_ocpd'], provides=['wire_works'])(check_wire_against_ocpd)
                        )

In [ ]:
results = graph(**{'ccc_count': 3, 'table': nec.ccc_count_derate,
                   'amb_temp': 30, 'wire_insulation_temp': 90,
                   'current': 60, 'parallel_conductors': 1,
                   'ocpd_derate': 0.8,
                   'wire_material': 'Cu', 'terminal_temp_rating': 75,
                   'voltage_type':'DC'})

In [ ]:
for key, val in results.items():
    print('{}: {}'.format(key, val))

In [ ]:
graph.plot(filename='base_wire_size.pdf')

## Recreate spreadsheet approach that has user input wire size and checks if it is large enough

This DAG re-creates the calculation stream from the spreadsheet that we agree results in the correct wire size. This DAG takes the wire size as an input, which is the most signifcant difference from the above DAG. 

In [ ]:
check_wire = graphtik.compose('check_selected_wire_size',
                         graphtik.operation(name='ccc_derate', needs=['ccc_count', 'table'], provides=['ccc_derate'])(osd.lookup),
                         graphtik.operation(name='amb_temp_derate', needs=['amb_temp', 'wire_insulation_temp'], provides=['amb_derate'])(osd.get_ambient_temp_derate),
                         graphtik.operation(name='cond_of_use_derates', needs=['ccc_derate', 'amb_derate'], provides=['total_derate'])(operator.mul),
                         graphtik.operation(name='current_per_conductor', needs=['current', 'parallel_conductors'], provides=['parallel_current'])(operator.mul),
                         graphtik.operation(name='terminal_check_current', needs=['parallel_current', 'ocpd_derate'], provides=['terminal_check_current'])(operator.truediv),
                         graphtik.operation(name='cond_use_current', needs=['parallel_current', 'total_derate'], provides=['cond_use_check_current'])(operator.truediv),
                         graphtik.operation(name='get_design_current', needs=['terminal_check_current', 'cond_use_check_current'], provides=['design_current'])(max),
                         graphtik.operation(name='check_terminals', needs=['design_current', 'wire_size', 'wire_material', 'terminal_temp_rating'], provides=['terminals_passing'])(check_wire_ampacity),
                         graphtik.operation(name='get_ocpd', needs=['current', 'voltage_type', 'ocpd_derate'], provides=['ocpd'])(osd.get_ocpd),
                         graphtik.operation(name='cable_sizing_ocpd', needs=['ocpd'], provides=['cable_sizing_ocpd'])(osd.get_cable_sizing_ocpd),
                         graphtik.operation(name='ocpd_check', needs=['cable_sizing_ocpd', 'wire_size', 'wire_material', 'wire_insulation_temp'], provides=['derated_cable_ampacity_exceeds_ocpd'])(check_wire_ampacity),
                         graphtik.operation(name='check_all', needs=['terminals_passing', 'derated_cable_ampacity_exceeds_ocpd'], provides=['cable_passing'])(operator.and_)
                        )

In [ ]:
check_wire_results = check_wire(**{'ccc_count': 3, 'table': nec.ccc_count_derate,
                                   'amb_temp': 34, 'wire_insulation_temp': 90,
                                   'current': 127.2, 'parallel_conductors': 1,
                                   'ocpd_derate': 0.8,
                                   'wire_size':'4/0',
                                   'wire_material': 'Al', 'terminal_temp_rating': 75,
                                   'voltage_type':'AC'})

In [ ]:
for key, val in check_wire_results.items():
    print('{}: {}'.format(key, val))

In [ ]:
check_wire.plot()

### Put calc in loop to get base wire size

This section of the notebook puts the above calculation inside a loop to find the base wire size without manually iterating. For this example, this DAG gives the same results as the excel spreadsheet.
- A starting wire size is looked up using the un-modified current, no derates, and the wire insulation temperature rating. For this example this approach returns a wire size that is below the base wire size, but only by a size or two. The intent is to limit iterations without missing the base size. Testing of many scenarios is necessary to be sure this doesn't ever return a wire size larger than the base size.
- The calculation is then re-run with larger wires until the wire passes the terminal/cont. duty and the cond of use vs ocpd checks.
- This could be improved by seperating it into two DAGs because the wire size is not an input until the second half of the calculation.

I think this approach is viable as the basis for further development as opposed to working out a calculation stream that determines the base wire size directly without the loop.  The loop only needs to be run once when initializing a table and then the base wire size can be stored.  Then the same DAG can be re-run whenver the user changes the cable size. (Usually for VD. Should users be allowed to input a wire size below the min for ampacity? The flexability might be nice and people could use that to find the pass/fail breakpoint themselves, which is reassuring, but this also allows users to accidently leave a cable size that is not code compliant in their table.  Maybe allow it and warn strongly?).

In [ ]:
inputs = {'ccc_count': 3, 'table': nec.ccc_count_derate,
          'amb_temp': 34, 'wire_insulation_temp': 90,
          'current': 127.2, 'parallel_conductors': 1,
          'ocpd_derate': 0.8,
          'wire_size':'4/0',
          'wire_material': 'Al', 'terminal_temp_rating': 75,
          'voltage_type':'AC'}

In [ ]:
begin_wire_size = osd.lookup(inputs['current'], nec.cable_ampacity_310_16[inputs['wire_material']][inputs['wire_insulation_temp']], keys=False)

In [ ]:
inputs['wire_size'] = begin_wire_size

In [ ]:
inputs['wire_size']

In [ ]:
wire_sizes = list(nec.cable_ampacity_310_16[inputs['wire_material']][inputs['wire_insulation_temp']].keys())

In [ ]:
begin_wire_size_ix = wire_sizes.index(begin_wire_size)

In [ ]:
for next_wire_size in wire_sizes[begin_wire_size_ix:]:
    inputs['wire_size'] = next_wire_size
    check_wire_results = check_wire(**inputs)
    if check_wire_results['cable_passing']:
        base_cable_size = next_wire_size
        break

In [ ]:
base_cable_size

## Check applying derates to ocpd size instead of cable

Is there a direct way to obtain the solution without inputting a wire size and then manually or automatically bumping wire sizes to find the solution.
Would need to apply conditions of use to the next lowest ocpd size, but that might cause incorrect breakpoints in selected sizes?

In [ ]:
uprated_ocpd = 90 / 0.786
uprated_ocpd

In [ ]:
get_wire_size(uprated_ocpd, 'Cu', 90)

In [ ]:
nec.cable_ampacity_310_16['Cu'][90]